In [5]:
import random

def calc_bmi(h,w):
    bmi=w / (h/100) **2
    if bmi < 18.5: return '1'
    if bmi < 25: return '2'
    return '3'

fp=open('d:/data/bmi/bmi.csv','w',encoding='utf-8') #쓰기 모드 파일
fp.write('height,weight,label\n') #파일의 헤더 기록
cnt={'1':0, '2':0, '3':0}
for i in range(20000):
    h=random.randint(120,200) #120~200 사이의 정수 난수
    w=random.randint(35,80)
    label=calc_bmi(h,w)
    cnt[label] += 1 #카운트 증가
    fp.write('{0},{1},{2}\n'.format(h,w,label))
    
fp.close() #파일닫기
print(cnt,'건의 데이터 파일이 생성되었습니다.')
    

{'1': 6412, '2': 5929, '3': 7659} 건의 데이터 파일이 생성되었습니다.


In [6]:
import pandas as pd
df=pd.read_csv('d:/data/bmi/bmi.csv')
df.head()

,height,weight,label
0,129,65,3
1,143,42,2
2,179,70,2
3,144,62,3
4,146,48,2


In [7]:
train_cols=df.columns[0:2]
X=df[train_cols]
y=df['label']

In [8]:
df['label'].value_counts()

3    7659
1    6412
2    5929
Name: label, dtype: int64

In [10]:
train_cols=['height','weight']
X=df[train_cols] #독립변수
y=df['label'] #종속변수

In [11]:
#언더샘플링
from imblearn.under_sampling import RandomUnderSampler
X_sample,y_sample=RandomUnderSampler(random_state=0).fit_sample(X,y)
X_samp=pd.DataFrame(data=X_sample,columns=train_cols)
y_samp=pd.DataFrame(data=y_sample,columns=['label'])
df2=pd.concat([X_samp,y_samp],axis=1)

In [12]:
df2['label'].value_counts() #샘플링 후의 클래스별 카운트

3    5929
2    5929
1    5929
Name: label, dtype: int64

In [13]:
X=X_samp[['height','weight']]
y=y_samp['label']

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#학습용:검증용 8:2로 구분
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=10)
model=LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
print(model.score(X_train,y_train)) #학습용 정확도
print(model.score(X_test,y_test)) #검증용 정확도

0.9825005270925574
0.9817313097245643


In [22]:
from sklearn.metrics import confusion_matrix
pred=model.predict(X_test) #예측값
print(pred[:10]) #예측값
print(y_test.values[:10]) #실제값
cm=confusion_matrix(y_test,pred) #실제값과 예측값의 비교
cm

[3 3 3 3 3 2 1 3 3 1]
[3 3 3 3 3 2 1 3 3 1]


array([[1165,   13,    0],
       [  20, 1158,   16],
       [   0,   16, 1170]], dtype=int64)

In [20]:
df_cm=pd.DataFrame(cm,index=['저체중','정상','과체중'],
                  columns=['저체중','정상','과체중'])
df_cm

,저체중,정상,과체중
저체중,1165,13,0
정상,20,1158,16
과체중,0,16,1170
